In [6]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
import dask.dataframe as daf

In [5]:
xr.DataArray( data = np.arange(1,10).reshape(3,3),
             dims = ['x','y'])

<xarray.DataArray (x: 3, y: 3)>
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])
Dimensions without coordinates: x, y

In [7]:
def ndwi_band(raster):   # CHECK BAND #s THEYRE WRONG
    x = raster.sel(band=2).astype('int16') 
    y = raster.sel(band=4).astype('int16') 
    return (x-y) / (x+y)

def ndvi_band(raster):
    x = raster.sel(band=4).astype('int16') 
    y = raster.sel(band=2).astype('int16') 
    return (x-y) / (x+y)

In [8]:
data = np.array([np.ones((3,3)),
                 np.ones((3,3))*2,
                 np.ones((3,3))*3,
                 np.ones((3,3))*4])

data = np.array([
                np.arange(1,10).reshape(3,3)*2+1,
                np.arange(1,10).reshape(3,3),
                np.arange(1,10).reshape(3,3),
                np.arange(1,10).reshape(3,3)])

test = xr.DataArray(data = data,
                   dims = ['band','x','y'],
                   #coords = {'band' : ['r','g','b','nir']})
                    coords = {'band' : [1,2,3,4]})
test
#mask = test.where(test<5)
#test * mask

<xarray.DataArray (band: 4, x: 3, y: 3)>
array([[[ 3,  5,  7],
        [ 9, 11, 13],
        [15, 17, 19]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]]])
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: x, y

In [9]:
ndvi = xr.DataArray(data = [ndvi_band(test)],
                   dims = ['band','x','y'],
                   coords = {'band' : [5]})

X = xr.concat([test,ndvi], dim='band')
X.T 

<xarray.DataArray (y: 3, x: 3, band: 5)>
array([[[ 3.,  1.,  1.,  1.,  0.],
        [ 9.,  4.,  4.,  4.,  0.],
        [15.,  7.,  7.,  7.,  0.]],

       [[ 5.,  2.,  2.,  2.,  0.],
        [11.,  5.,  5.,  5.,  0.],
        [17.,  8.,  8.,  8.,  0.]],

       [[ 7.,  3.,  3.,  3.,  0.],
        [13.,  6.,  6.,  6.,  0.],
        [19.,  9.,  9.,  9.,  0.]]])
Coordinates:
  * band     (band) int64 1 2 3 4 5
Dimensions without coordinates: y, x

In [10]:
# apply this mask to pixels to keep those with ndwi below thresh
ndwi_thresh = 0  # keeps everything below this thresh
ndwi = ndwi_band(test)
ndwi
ndwi_mask = ndwi.where(ndwi>ndwi_thresh)#.isnull()
ndwi_mask

<xarray.DataArray (x: 3, y: 3)>
array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])
Dimensions without coordinates: x, y

In [18]:
np.array([0, 0, 1]).nonzero()[0]

array([2])

In [8]:
ndwi_mask.dropna(dim = 'x')

<xarray.DataArray (x: 0, y: 3)>
array([], shape=(0, 3), dtype=float64)
Dimensions without coordinates: x, y

In [5]:
# apply this mask to pixels to keep those with ndvi above thresh
ndvi_thresh = 0  # keeps everything below this thresh
ndvi = ndvi_band(test)
#ndvi
ndvi_mask = ndvi.where(ndvi<ndvi_thresh)#.isnull()
ndvi_mask

ndvi_mask * ndwi_mask * test

<xarray.DataArray (x: 3, y: 3, band: 4)>
array([[[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]],

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]],

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]])
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: x, y

In [3]:
pixels = test.stack(z=("x", "y")).drop_vars('z').T
pixels

<xarray.DataArray (z: 9, band: 4)>
array([[1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.]])
Coordinates:
  * band     (band) <U3 'r' 'g' 'b' 'nir'
Dimensions without coordinates: z

In [ ]:
pixels.isnull

In [ ]:
pixels.sel(band=1)

In [4]:
da_pixels = daf.from_array(pixels)
da_pixels

,0,1,2,3
npartitions=1,,,,
0,float64,float64,float64,float64
8,...,...,...,...


In [5]:
da_pixels.compute()

,0,1,2,3
0,1.0,2.0,3.0,4.0
1,1.0,2.0,3.0,4.0
2,1.0,2.0,3.0,4.0
3,1.0,2.0,3.0,4.0
4,1.0,2.0,3.0,4.0
5,1.0,2.0,3.0,4.0
6,1.0,2.0,3.0,4.0
7,1.0,2.0,3.0,4.0
8,1.0,2.0,3.0,4.0


In [6]:
### OTHER TRIALS

In [ ]:
def ndwi_band(raster):   # CHECK BAND #s THEYRE WRONG
    x = raster.sel(band=1).astype('int16') 
    y = raster.sel(band=3).astype('int16') 
    return (x-y) / (x+y)

def ndvi_band(raster):
    x = raster.sel(band=4).astype('int16') 
    y = raster.sel(band=1).astype('int16') 
    return (x-y) / (x+y)

In [ ]:
test = xr.DataArray(data = np.array([
                    np.arange(1,10).reshape(3,3)*2+1,
                    np.arange(1,10).reshape(3,3),
                    np.arange(1,10).reshape(3,3),
                    np.arange(1,10).reshape(3,3)]),
                   dims = ['band','x','y'],
                   coords = {'band' : [1,2,3,4]})
test
#mask = test.where(test<5)
#test * mask

In [ ]:
ndwi = ndwi_band(test)
ndwi

In [ ]:
ndwi_mask = ndwi.where(ndwi <0.37)
ndwi_mask

In [ ]:
ndwi_mask = ndwi_mask.isnull() 
ndwi_mask

In [ ]:
not_water = test * ndwi_mask
not_water

In [ ]:
ndvi = ndvi_band(not_water)
ndvi 

In [ ]:
ndwi_mask = ndvi.where(ndvi >-0.36).isnull()
ndwi_mask